In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [10]:
# 1️⃣ Filter Users with >= 200 Ratings
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index
df_ratings = df_ratings[df_ratings['user'].isin(active_users)]

# 2️⃣ Filter Books with >= 100 Ratings
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index
df_ratings = df_ratings[df_ratings['isbn'].isin(popular_books)]

# 3️⃣ Create User-Book Pivot Table
user_book_matrix = df_ratings.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

# 4️⃣ Map ISBN to Book Title
book_titles = df_books.set_index('isbn')['title']

# 5️⃣ Fit Nearest Neighbors Model
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_book_matrix.values)

# 6️⃣ Define get_recommends() Function
def get_recommends(book_title):
    try:
        isbn = df_books[df_books['title'] == book_title]['isbn'].values[0]
    except IndexError:
        return f"Book '{book_title}' not found in dataset."

    try:
        book_idx = user_book_matrix.index.tolist().index(isbn)
    except ValueError:
        return f"Book '{book_title}' not found in pivot table."

    distances, indices = model.kneighbors(
        user_book_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6
    )

    recommended_books = []
    for i in range(1, 6):
        neighbor_isbn = user_book_matrix.index[indices[0][i]]
        neighbor_title = book_titles[neighbor_isbn]
        distance = distances[0][i]
        recommended_books.append([neighbor_title, distance])

    return [book_title, recommended_books]

# 7️⃣ Test the Function
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")


"Book 'The Queen of the Damned (Vampire Chronicles (Paperback))' not found in pivot table."

In [11]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [13]:
def get_recommends(book_title):
    try:
        isbn = df_books[df_books['title'] == book_title]['isbn'].values[0]
    except IndexError:
        return f"Book '{book_title}' not found in dataset."

    try:
        book_idx = user_book_matrix.index.tolist().index(isbn)
    except ValueError:
        return f"Book '{book_title}' not found in pivot table."

    distances, indices = model.kneighbors(
        user_book_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6
    )

    recommended_books = []
    for i in range(1, 6):
        neighbor_isbn = user_book_matrix.index[indices[0][i]]
        neighbor_title = book_titles[neighbor_isbn]
        distance = distances[0][i]
        recommended_books.append([neighbor_title, distance])

    return [book_title, recommended_books]
